In [3]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification", model="MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli")
sequence_to_classify = "Angela Merkel is a politician in Germany and leader of the CDU"
candidate_labels = ["politics", "economy", "entertainment", "environment"]
output = classifier(sequence_to_classify, candidate_labels, multi_label=False)
print(output)

Device set to use mps:0
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'sequence': 'Angela Merkel is a politician in Germany and leader of the CDU', 'labels': ['politics', 'economy', 'environment', 'entertainment'], 'scores': [0.9823215007781982, 0.007280238438397646, 0.005891914013773203, 0.004506314639002085]}


In [1]:
from datasets import load_dataset

ds = load_dataset("wikimedia/wikipedia", "20231101.en", streaming=True)

print(ds)

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

IterableDatasetDict({
    train: IterableDataset({
        features: ['id', 'url', 'title', 'text'],
        num_shards: 41
    })
})


In [4]:
# Access the first element of the IterableDataset
first_element = next(iter(ds["train"]))
print(first_element)

{'id': '12', 'url': 'https://en.wikipedia.org/wiki/Anarchism', 'title': 'Anarchism', 'text': 'Anarchism is a political philosophy and movement that is skeptical of all justifications for authority and seeks to abolish the institutions it claims maintain unnecessary coercion and hierarchy, typically including nation-states, and capitalism. Anarchism advocates for the replacement of the state with stateless societies and voluntary free associations. As a historically left-wing movement, this reading of anarchism is placed on the farthest left of the political spectrum, usually described as the libertarian wing of the socialist movement (libertarian socialism).\n\nHumans have lived in societies without formal hierarchies long before the establishment of states, realms, or empires. With the rise of organised hierarchical bodies, scepticism toward authority also rose. Although traces of anarchist ideas are found all throughout history, modern anarchism emerged from the Enlightenment. During

In [10]:
sentences = first_element["text"].split("\n")
print(sentences[0])

Anarchism is a political philosophy and movement that is skeptical of all justifications for authority and seeks to abolish the institutions it claims maintain unnecessary coercion and hierarchy, typically including nation-states, and capitalism. Anarchism advocates for the replacement of the state with stateless societies and voluntary free associations. As a historically left-wing movement, this reading of anarchism is placed on the farthest left of the political spectrum, usually described as the libertarian wing of the socialist movement (libertarian socialism).


In [13]:
text_to_classify = sentences[0]
labels = ["sport", "not sport"]
output = classifier(text_to_classify, labels, multi_label=False)
print(output['labels'])
print(output['scores'])

['not sport', 'sport']
[0.9894245266914368, 0.010575489141047001]


In [50]:
text_sport = "Aberdeen is a city in Scotland."
labels = ["sport", "other"]
output = classifier(text_sport, labels, multi_label=False)
print(output['labels'])
print(output['scores'])

['sport', 'other']
[0.5111673474311829, 0.48883265256881714]


In [23]:
text_sport_fr = "La vie est belle."
labels = ["sport", "not sport"]
output = classifier(text_sport_fr, labels, multi_label=False)
print(output['labels'])
print(output['scores'])

['not sport', 'sport']
[0.8859437704086304, 0.11405624449253082]


In [22]:
text_sport_zh = "这是一棵树"
labels = ["运动", "非运动"]
output = classifier(text_sport_zh, labels, multi_label=False)
print(output['labels'])
print(output['scores'])

['非运动', '运动']
[0.9101300239562988, 0.08987002074718475]


In [39]:
empty_text = " "
labels = ["sport", "not sport"]
output = classifier(empty_text, labels, multi_label=False)
print(output['labels'])
print(output['scores'])

['not sport', 'sport']
[0.5762802362442017, 0.42371973395347595]


In [24]:
from transformers import pipeline

CLASSIFIER_MODEL = "MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli"
MISSION = "zero-shot-classification"
classifier = pipeline(MISSION, model=CLASSIFIER_MODEL)

def classify_text(text, classifier, labels):
    output = classifier(text, labels, multi_label=False)
    if output['labels'][0] == labels[0]:
        return True
    else:
        return False

Device set to use mps:0


In [25]:
from datasets import load_dataset

ds_en = load_dataset("wikimedia/wikipedia", "20231101.en", streaming=True)
ds_fr = load_dataset("wikimedia/wikipedia", "20231101.fr", streaming=True)
ds_zh = load_dataset("wikimedia/wikipedia", "20231101.zh", streaming=True)

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/17 [00:00<?, ?it/s]

In [45]:
from itertools import islice
from tqdm import tqdm
def dataset_filter(ds, classifier, labels, max_length=100):
    filtered_ds = []
    for element in tqdm(islice(ds, max_length)):
        text = element["text"].split("\n")[0]
        if text and classify_text(text, classifier, labels):
            filtered_ds.append(element)
    return filtered_ds

In [48]:
ds_sport = []
labels_en = ["sport", "other"]
ds_sport = dataset_filter(ds_en["train"], classifier, labels_en, 100)

print(len(ds_sport))

100it [00:58,  1.71it/s]

5


In [49]:
print(ds_sport)

[{'id': '595', 'url': 'https://en.wikipedia.org/wiki/Andre%20Agassi', 'title': 'Andre Agassi', 'text': 'Andre Kirk Agassi ( ; born April 29, 1970) is an American former world No. 1 tennis player. He is an eight-time major champion and an Olympic gold medalist, as well as a runner-up in seven other majors. Agassi is widely considered one of the greatest tennis players of all time.\n\nAgassi is the second of five men to achieve the career Grand Slam in the Open Era and the fifth of eight overall to make the achievement. He is also the first of two men to achieve the career Golden Slam (career Grand Slam and Olympic gold medal), as well as the only man to win a career Super Slam (career Grand Slam, plus the Olympic gold medal and the year-end championships).\n\nAgassi was the first man to win all four singles majors on three different surfaces (hard, clay and grass), and remains the most recent American man to win the French Open (in 1999) and the Australian Open (in 2003). He also won 17